In [ ]:
参考（ここに書かれていることをやった）
https://deepage.net/deep_learning/2016/11/07/convolutional_neural_network.html
http://tecmemo.wpblog.jp/category/machine-learning/deep-learning/tensorflow/

（wやxの定義の仕方の理解に役立つ）
https://endoyuta.com/2017/01/18/tensorflow-%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF%E6%BC%94%E7%AE%97%E3%81%AE%E9%96%A2%E6%95%B0-tf-nn-conv2d/
（震探データに応用する際に重要な，channelの扱いについて書かれている)
https://qiita.com/YusukeSuzuki@github/items/0764d15b9d0b97ec1e16
    

In [35]:
import numpy as np
import tensorflow as tf

######     設定部     #########
#conv1
filter_height_conv1=5#filter縦長さ
filter_width_conv1=5#filter横長さ
num_of_conv1_opchannels=32
dx_conv1=1#横方向ストライド
dy_conv1=1#縦方向ストライド
#pool1
k_height_pool1=2
k_width_pool1=2
dy_pool1=2
dx_pool1=2
#conv2
filter_height_conv2=5#filter縦長さ
filter_width_conv2=5#filter横長さ
num_of_conv2_opchannels=64
dx_conv2=1
dy_conv2=1
#pool1
k_height_pool2=2
k_width_pool2=2
dy_pool2=2
dx_pool2=2
#fc1
num_of_op_nodes_fc1=1024
#fc2
num_of_op_nodes_fc2=10

learning_rate=0.00001
num_of_steps=20000
batch_size=50
#############################
#cnnは，普通のニューラルネットワークと基本的には同じ構造．
#ただ，"node（数字）"の部分が"channel（テンソル）"になっている．
#フィルタサイズ(a,b), input-channelの要素数i, output-channelの要素数jというのは，
#wというvariablesは，i行j列の，一個一個の要素がサイズ(a,b)のテンソル（＝フィルタ）である，というイメージ．

#MNISTデータのダウンロード
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_dataset/", one_hot=True)


def conv2d(x,w,b,dy,dx):
    op_conv2d=tf.nn.conv2d(x, w, strides=[1,dy,dx,1],padding='SAME')+b
    #nn.cnnv2dは，フィルター内のxとwの対応する要素同士を掛け，その和を返す．
    #stridesは，[1, dy, dx, 1]となっていて，縦方向にdy，横方向にdxピクセル幅でフィルタを動かす
    #paddding=sameは，フィルター内に一か所でももともとのデータがある場所なら，残りをゼロで埋める
    op_conv2d=tf.nn.relu(op_conv2d)#reluは活性化関数．sigmoidみたいなもん．o以上の値に直す
    return op_conv2d

def max_pool(x,k_height,k_width,dy, dx):
    op_pool=tf.nn.max_pool(x, ksize=[1, k_height, k_width, 1],
                          strides=[1, dy, dx, 1], padding="SAME")
    return op_pool    

X=tf.placeholder("float", [None, 784])#input dataの[batch，縦，横，チャンネル数]
X_image=tf.reshape(X, [-1, 28, 28,1])
t=tf.placeholder("float", [None, 10])

#conv1
w_conv1=tf.Variable(tf.random_normal([filter_height_conv1, filter_width_conv1, 1, num_of_conv1_opchannels]))
#wはフィルターテンソル．[縦，横，入力チャネル数，出力チャネル数]
#入力チャネル（=nodeな気がする）数は，白黒なら1，RGBなら3のように，ある場所がもつ要素の数
#tf.nn.conv2dでは，input channnelが複数ある場合，output channelにはそれらの合計値が出される．
#出力チャネル数は，num_of_hidden_nodes的なものなので，自由に決めてよい（2の乗数が一般的）
b_conv1=tf.Variable(tf.random_normal([num_of_conv1_opchannels]))
#filter w+b を，位置普遍性を保つように最適化していくというのがコンセプトなので，
#w及びbは，input_channel*output_channel個づつになる．
#ただし，wは5*5などの形を持っているため，このようになる．．
op_conv1=conv2d(X_image, w_conv1,b_conv1,dy_conv1, dx_conv1)

#pool1
op_pool1=max_pool(op_conv1,k_height_pool1,k_width_pool1,dy_pool1,dx_pool1)

#conv2
w_conv2=tf.Variable(tf.random_normal([filter_height_conv2, filter_width_conv2,
                                      num_of_conv1_opchannels, num_of_conv2_opchannels]))
b_conv2=tf.Variable(tf.random_normal([num_of_conv2_opchannels]))
op_conv2=conv2d(op_pool1, w_conv2,b_conv2,dy_conv2, dx_conv2)

#pool2
op_pool2=max_pool(op_conv2, k_height_pool2, k_width_pool2, dy_pool2, dx_pool2)

#全結合層1
op_pool2_height=round(28/dy_conv1/dy_conv2/dy_pool1/dy_pool2)
op_pool2_width=round(28/dx_conv1/dx_conv2/dx_pool1/dx_pool2)
op_pool2=tf.reshape(op_pool2,[-1, op_pool2_height*op_pool2_width*num_of_conv2_opchannels])
w_fc1=tf.Variable(tf.random_normal([op_pool2_height*op_pool2_width*num_of_conv2_opchannels, num_of_op_nodes_fc1]))
b_fc1=tf.Variable(tf.random_normal([num_of_op_nodes_fc1]))
op_fc1=tf.nn.relu(tf.matmul(op_pool2, w_fc1)+b_fc1)

#全結合層2
w_fc2=tf.Variable(tf.random_normal([num_of_op_nodes_fc1, num_of_op_nodes_fc2]))
b_fc2=tf.Variable(tf.random_normal([num_of_op_nodes_fc2]))
op_fc2=tf.nn.softmax(tf.matmul(op_fc1, w_fc2)+b_fc2)

cross_entropy=-tf.reduce_sum(t*tf.log(tf.clip_by_value(op_fc2, 1e-10, 1.0)))#tのplaceholderの定義を必要とするのは実際にはここ以降

#optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

#for tensorboard
#tf.summary.scalar("cross_entropy", cross_entropy)
#summary_op=tf.summary.merge_all()

#評価用
correct_prediction=tf.equal(tf.argmax(op_fc2,1), tf.argmax(t,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    #summary_writer=tf.summary.FileWriter('.\logs/180115', sess.graph)
    sess.run(tf.global_variables_initializer())
    for step in range (num_of_steps):
        x_batch,t_batch=mnist.train.next_batch(batch_size)

        sess.run(optimizer, feed_dict={X:x_batch, t:t_batch})
        if step % 100 == 0:
            result=sess.run(accuracy, feed_dict={X: mnist.test.images, t: mnist.test.labels})
            print('step={}/accuracy={}'.format(step, result))
            
            #wc1=sess.run(w_conv1)
            #print('w_conv1=\n',wc1)
            
            #summary=sess.run(summary_op,feed_dict={X: mnist.test.images, t: mnist.test.labels})
            #summary_writer.add_summary(summary,step)
            
    result=sess.run(accuracy, feed_dict={X: mnist.test.images, t: mnist.test.labels})
    print("final_accuracy=\n",result)

Extracting MNIST_dataset/train-images-idx3-ubyte.gz
Extracting MNIST_dataset/train-labels-idx1-ubyte.gz
Extracting MNIST_dataset/t10k-images-idx3-ubyte.gz
Extracting MNIST_dataset/t10k-labels-idx1-ubyte.gz
step=0/accuracy=0.1876000016927719
step=100/accuracy=0.20190000534057617
step=200/accuracy=0.10520000010728836
step=300/accuracy=0.10520000010728836
step=400/accuracy=0.10520000010728836
step=500/accuracy=0.10520000010728836
step=600/accuracy=0.10520000010728836
step=700/accuracy=0.10520000010728836
step=800/accuracy=0.10520000010728836


KeyboardInterrupt: 

In [22]:
#確認variableの定義の仕方
import tensorflow as tf
a=tf.Variable(tf.random_normal([2, 2, 3, 4]))
print(a)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a))

<tf.Variable 'Variable_133:0' shape=(2, 2, 3, 4) dtype=float32_ref>
[[[[ 0.37061283 -1.3806987   0.8084075   0.9719897 ]
   [-0.2629388   1.7086697   1.2886996   0.6605297 ]
   [-1.4800066   1.0990572   0.53192234 -0.8736463 ]]

  [[ 1.7176734   0.13660447 -0.7850778  -1.5814241 ]
   [-1.7370864   1.08038    -1.039911    1.7904675 ]
   [-0.38866213 -0.07242326 -1.083528   -0.64173394]]]


 [[[-0.8982141   1.3345892  -0.62835425 -0.15981498]
   [-0.0722959  -0.22174725 -0.6626167  -0.2308909 ]
   [ 0.1832864   0.7508374   0.60154486  1.9067985 ]]

  [[-1.3113804  -0.37368625  1.1420406   0.70210665]
   [ 1.124935    0.59519947  0.9509193   0.44264832]
   [ 1.477632    0.7944328   1.5487107  -0.18196186]]]]


In [25]:
#確認用2: 切り上げ
print(round(12/4))

3


In [ ]:
# -*- coding: utf-8 -*-
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
 
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
 
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
   
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
 
# ニューラルネットワークを計算グラフで作成する
# 第1層 (入力層)
x = tf.placeholder("float", [None, 784])
 
# 形状変更
x_image = tf.reshape(x, [-1, 28, 28, 1])
 
# 第2層 (畳み込み層)
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
y_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
 
# 第3層 (プーリング層)
y_pool1 = max_pool_2x2(y_conv1)
 
# 第4層 (畳み込み層)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
y_conv2 = tf.nn.relu(conv2d(y_pool1, W_conv2) + b_conv2)
 
# 第5層 (プーリング層)
y_pool2 = max_pool_2x2(y_conv2)
 
# 形状変更
y_pool2_flat = tf.reshape(y_pool2, [-1, 7 * 7 * 64])
 
# 第6層 (全結合層)
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
y_fc1 = tf.nn.relu(tf.matmul(y_pool2_flat, W_fc1) + b_fc1)
 
# 第7層 (全結合層)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y = tf.nn.softmax(tf.matmul(y_fc1, W_fc2) + b_fc2)
 
# 損失関数を計算グラフを作成する
t = tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(t * tf.log(y))
 
# 次の(1)、(2)を行うための計算グラフを作成する。
# (1) 損失関数に対するネットワークを構成するすべての変数の勾配を計算する。
# (2) 勾配方向に学習率分移動して、すべての変数を更新する。
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
 
# 初期化を行うための計算グラフを作成する。
init = tf.global_variables_initializer()
 
# テストデータに対する正答率を計算するための計算グラフを作成する。
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
 
# MNIST 入力データ
mnist = input_data.read_data_sets("data/", one_hot=True)
 
# セッションを作成して、計算グラフを実行する。
with tf.Session() as sess:
   
    # 初期化を実行する。
    sess.run(init)
   
    # 学習を実行する。
    for i in range(20000):
        x_batch, t_batch = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x: x_batch, t: t_batch})
         
        if i % 100 == 0:
            result = sess.run(accuracy, feed_dict={x: mnist.test.images, t: mnist.test.labels})
            print(result)
 
    result = sess.run(accuracy, feed_dict={x: mnist.test.images, t: mnist.test.labels})
    print("accuracy:", result)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
